In [79]:
import pandas as pd
import numpy as np

#Methods
from sklearn.model_selection import train_test_split
from interpret.glassbox import ExplainableBoostingClassifier

#helper functions
import helper
import importlib
importlib.reload(helper)



<module 'helper' from '/home/bart/AI3/project/XAI_models/helper.py'>

In [80]:
#import breast dataset
X = pd.read_csv("./datasets/cleaned/breast_X.csv")
X = X.drop("Unnamed: 0", axis=1)
y = pd.read_csv("./datasets/cleaned/breast_y.csv")
y = y.drop("Unnamed: 0", axis=1)
X.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [81]:
features_types_df = pd.read_csv(f"datasets/cleaned/datatypes/breast.csv")

feature_inidices = list(map(int, list(features_types_df)))
features_names = list(features_types_df.T[0])
features_types = list(map(int, list(features_types_df.T[1])))

In [82]:
#define data containers for features
#input_queue: indices of features to be check
#output_queue: indices of checked features (order is coresponding to given loss)
#data_losses: data container for each step losses storing
input_queue = pd.Series(feature_inidices, dtype=int)
output_queue = pd.Series([], dtype=int)
run_losses = pd.Series([], dtype=float)

In [83]:
from interpret.glassbox import ExplainableBoostingClassifier
ebm = ExplainableBoostingClassifier()

In [84]:
#define data variable for initial loss
# initial_index = [np.random.choice(input_queue)]
# initial_index = np.random.choice(input_queue)
initial_index = 0
test_df = helper.create_data_frame_for_feat(helper.get_output_col_name([initial_index], pd.Series(features_names)), X)
result, name = helper.calculate_loss_for_single_feat(test_df,
                                                     y[y.columns[0]],
                                                     initial_index, features_names, features_types, ebm)

In [85]:
initial_error = result
initial_name  = name

In [86]:
name

'radius_mean'

In [87]:
len(feature_inidices)

30

In [88]:
losses_vector = np.zeros(len(feature_inidices))
for index in feature_inidices:
  test_df = helper.create_data_frame_for_feat(helper.get_output_col_name([index], pd.Series(features_names)), X)
  result, name = helper.calculate_loss_for_single_feat(test_df,
                                                      y[y.columns[0]],
                                                      index, features_names, features_types, ebm)
  losses_vector[index] = result
  print(name)

radius_mean
texture_mean
perimeter_mean
area_mean
smoothness_mean
compactness_mean
concavity_mean
concave points_mean
symmetry_mean
fractal_dimension_mean
radius_se
texture_se
perimeter_se
area_se
smoothness_se
compactness_se
concavity_se
concave points_se
symmetry_se
fractal_dimension_se
radius_worst
texture_worst
perimeter_worst
area_worst
smoothness_worst
compactness_worst
concavity_worst
concave points_worst
symmetry_worst
fractal_dimension_worst


In [89]:
losses_vector

array([ 1.95042502, 10.40227042,  3.5757805 ,  1.62535418, 10.40226912,
        6.82649121,  4.55099301,  2.60056747,  9.42705556,  9.75212588,
        4.87606437, 10.40226677,  4.87606463,  2.92563779, 10.72733787,
       10.40227224,  8.1267756 ,  8.45184644,  9.75212588, 10.72733995,
        1.62535418,  8.45184488,  1.95042528,  1.62535418,  5.8512774 ,
        7.15156023,  3.57578129,  3.25070915,  6.17634616,  9.10198525])

In [90]:
run_losses[0] = losses_vector

In [91]:
# get index of smallest loses feature
feature_selected_index = input_queue.iloc[run_losses[0].argmin()]
#pop index from input queue
input_queue.pop(feature_selected_index)
#add selected index to output_queue
output_queue = pd.concat([output_queue, pd.Series(feature_selected_index)])

In [92]:
input_queue

0      0
1      1
2      2
3      3
4      4
5      5
6      6
7      7
8      8
9      9
10    10
11    11
12    12
13    13
14    14
15    15
16    16
17    17
18    18
19    19
21    21
22    22
23    23
24    24
25    25
26    26
27    27
28    28
29    29
dtype: int64

In [93]:
output_queue

0    20
dtype: int64

In [94]:
def concat_data_indices(output_queue, input_queue):
  temp_list = list(output_queue)
  indices_list = []
  for index_input in input_queue:
    temp_list.append(index_input)
    indices_list.append(temp_list)
    temp_list = list(output_queue)
  return indices_list

In [95]:
lista_test = concat_data_indices(output_queue, input_queue)

In [96]:
lista_test[0]

[20, 0]

In [103]:
test_1 = helper.get_output_col_name(lista_test[0], pd.Series(features_names))

In [108]:
test_1

0    radius_worst
0     radius_mean
dtype: object

In [109]:
test_df_df = helper.create_data_frame_for_feat(test_1, X)

In [110]:
test_df_df

,radius_worst,radius_mean
0,25.380,17.99
1,24.990,20.57
2,23.570,19.69
3,14.910,11.42
4,22.540,20.29
...,...,...
564,25.450,21.56
565,23.690,20.13
566,18.980,16.60
567,25.740,20.60


In [121]:
X_train, X_test, y_train, y_test = train_test_split(test_df_df, y, test_size=0.3, random_state=1337, shuffle=True)

In [122]:
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=1337)


In [143]:
list(X_test)

['radius_worst', 'radius_mean']

In [139]:
ebm.fit(X_test[0], y_test)

KeyError: 0

In [137]:
    result, name = helper.calculate_loss_for_multi_feat(test_df_df,
                                                        y,
                                                        list(lista_test[j]),
                                                        features_names,
                                                        features_types,
                                                        ebm)

     radius_worst  radius_mean
0          25.380        17.99
1          24.990        20.57
2          23.570        19.69
3          14.910        11.42
4          22.540        20.29
..            ...          ...
564        25.450        21.56
565        23.690        20.13
566        18.980        16.60
567        25.740        20.60
568         9.456         7.76

[569 rows x 2 columns]
     diagnosis
0            1
1            1
2            1
3            1
4            1
..         ...
564          1
565          1
566          1
567          1
568          0

[569 rows x 1 columns]
ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(), []),
                                ('standardscaler', StandardScaler(),
                                 ['radius_worst', 'radius_mean'])])


ValueError: Shape of passed values is (398, 2), indices imply (398, 1)

In [ ]:
for i in range(len(input_queue)):
  losses_vector = np.zeros(len(input_queue))
  lista_test = concat_data_indices(output_queue, input_queue)

  for j in range(len(input_queue)):
    test_df = helper.create_data_frame_for_feat(helper.get_output_col_name(list(lista_test[j]), pd.Series(features_names)), X)
    # test_df = create_data_frame_for_feat(get_output_col_name(list(lista_test[j]), dataset_col_names), adult_df)
    # result, name = calculate_loss_for_multi_feat(test_df, y, list(lista_test[j]))
    result, name = helper.calculate_loss_for_multi_feat(test_df,
                                                        y[y.columns[0]],
                                                        list(lista_test[j]),
                                                        features_names,
                                                        features_types,
                                                        ebm)
    losses_vector[j] = result
  
  run_losses[i+1] = losses_vector
  # get index of smallest loses feature
  feature_selected_index = input_queue.iloc[run_losses[i+1].argmin()]
  input_queue.pop(feature_selected_index)
  #add selected index to output_queue
  output_queue = pd.concat([output_queue, pd.Series(feature_selected_index)])

     radius_worst  radius_mean
0          25.380        17.99
1          24.990        20.57
2          23.570        19.69
3          14.910        11.42
4          22.540        20.29
..            ...          ...
564        25.450        21.56
565        23.690        20.13
566        18.980        16.60
567        25.740        20.60
568         9.456         7.76

[569 rows x 2 columns]
0      1
1      1
2      1
3      1
4      1
      ..
564    1
565    1
566    1
567    1
568    0
Name: diagnosis, Length: 569, dtype: int64
ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(), []),
                                ('standardscaler', StandardScaler(),
                                 ['radius_worst', 'radius_mean'])])


ValueError: Shape of passed values is (398, 2), indices imply (398, 1)